In [64]:
import pandas as pd
import numpy as np
from pathlib import Path
from fastai.text import *

In [65]:
DATA_PATH = Path('data')
DATA_PATH

WindowsPath('data')

In [175]:
TMP_PATH = DATA_PATH/'tmp'
TMP_PATH.mkdir(exist_ok=True)

## Data

In [66]:
dataset = pd.read_json(DATA_PATH/'train.json')
dataset.head()

,comments,description,id,sql,sql_plain,title,url
0,[],,586912,"SELECT p.Title,p.score,p.ViewCount,p.AnswerCou...","SELECT \np.Title,p.score,p.ViewCount,p.AnswerC...",Repent 151-160,http://data.stackexchange.com/stackoverflow/re...
1,[This is test data for my question to make thi...,,54791,"CREATE TABLE #CustomerDB ( ID int NOT NULL, NA...",-- This is test data for my question to make t...,This is test data for my question to make thin...,http://data.stackexchange.com/stackoverflow/re...
2,[],,611565,select Id from Comments where lower(Text) like...,select Id\nfrom Comments \nwhere lower(Text) l...,Accept answer,http://data.stackexchange.com/stackoverflow/re...
3,[],,612812,"DECLARE @From DATETIME = convert(DATETIME, '##...","DECLARE @From DATETIME = convert(DATETIME, '##...",Questions with most views created within 3 mon...,http://data.stackexchange.com/stackoverflow/re...
4,"[Most Prolific Answerers (with score), Shows t...",,325114,"SELECT TOP 50 OwnerUserId as [User Link], COUN...",-- Most Prolific Answerers (with score)\n-- Sh...,Most Prolific Answers,http://data.stackexchange.com/stackoverflow/re...


In [146]:
SQL = dataset['sql']

In [147]:
## Transform the text to be tokenize readily
token_replaced = {',' : ' , ',
                 '#' : ' # ',
                 '@' : ' @ ',
                 '!' : ' ! ',
                 '.' : ' . ',
                 '%' : ' % ',
                 '?' : ' ? '}
    

In [148]:
def fixup(x):
    for bereplaced, replace in token_replaced.items():        
        x = x.replace(bereplaced, replace)
    return x

In [149]:
## Transform the text to be tokenize readily, as certain token has special meaning in SQL, ie. "." means the schema relation
token_replaced = {',' : ' , ',
                 '#' : ' # ',
                 '@' : ' @ ',
                 '!' : ' ! ',
                 '.' : ' . ',
                 '%' : ' % ',
                 '?' : ' ? ',
                 ')' : ' ) ',
                 '(' : ' ( ',
                 '=' : ' = ',
                 '/' : ' / ',
                 '*' : ' * '}
    


In [150]:
SQL = SQL.apply(fixup)

In [154]:
## Sanity Check
SQL.iloc[0]

In [156]:
SQL.iloc[1]

"CREATE TABLE  # CustomerDB  (  ID int NOT NULL ,  NAME nvarchar ( 50 )  NULL ,  COMPANY nvarchar ( 50 )  NULL ,  ADDR1 nvarchar ( 50 )  NULL ,   )  INSERT INTO  # CustomerDB ( ID ,  NAME ,  COMPANY ,  ADDR1 )  VALUES  ( 16242 , 'TOM E' , 'Company A' , '101 First RD' )  ,   ( 16241 , 'RONALD J' , 'Company B' , '12 Tenth AVE' )  ,   ( 16235 , 'KENNETH H' , '' , '12 Tenth AVE' )  ,   ( 16238 , 'MICHAEL H' , 'Company C' , '12 Tenth AVE' )  ,   ( 16243 , 'ANTONIO D' , 'Company D' , '264 Long ST STE 5' )  ,   ( 16237 , 'MICHAEL B' , 'Company E' , '264 Long ST STE 5' )  ,   ( 16234 , 'WALTER L' , 'Company F' , '73 North RD' )  ,   ( 16236 , 'CARL O' , 'Company G' , '73 North RD' )  ,   ( 16239 , 'MICHAEL S' , 'Company H' , '73 North RD' )  ,   ( 16240 , 'MICHAEL I' , 'Company I' , '73 North RD' )  SELECT c1 . id ,  c1 . name ,  c1 . company ,  c1 . addr1 FROM  # CustomerDB c1 LEFT OUTER JOIN  # CustomerDB c2 ON  ( c1 . addr1 =  c2 . addr1 )  GROUP BY c1 . addr1 ,  c1 . id ,  c1 . name ,  c1 

In [155]:
SQL.iloc[100]

"select a . month ,  total ,  unanswered ,  cast ( cast ( unanswered*1 . 0/total*100 as decimal ( 5 , 1 )  )  as varchar ( 5 )  )  + ' % ' as PercentUnanswered from  ( select Convert ( varchar ( 6 )  ,  creationdate ,  112 )  as month ,  count ( * )  as total from posts where posttypeid  =  1 group by Convert ( varchar ( 6 )  ,  creationdate ,  112 )  )  a ,   ( select Convert ( varchar ( 6 )  ,  creationdate ,  112 )  as month ,  count ( * )  as unanswered from posts where postTypeID  =  1 and id not in  (  select distinct ParentId from posts where PostTypeId  =  2 )  group by Convert ( varchar ( 6 )  ,  creationdate ,  112 )  )  b where a . month  =  b . month order by a . month"

In [160]:
sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL)) 
# sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL.iloc[:10])) # For Testing

In [158]:
tmp[0]

[' ',
 't_up',
 'select',
 'p',
 '.',
 'title',
 ',',
 'p',
 '.',
 'score',
 ',',
 'p',
 '.',
 'viewcount',
 ',',
 'p',
 '.',
 'answercount',
 ',',
 'p',
 '.',
 'commentcount',
 ',',
 't_up',
 'len',
 '(',
 'p',
 '.',
 'body',
 ')',
 'as',
 'ques_size',
 ',',
 'p',
 '.',
 'favoritecount',
 ',',
 'p',
 '.',
 'i',
 'd',
 ',',
 'u',
 '.',
 'reputation',
 'as',
 'user_repo',
 ',',
 'u',
 '.',
 'creationdate',
 'as',
 'user_join_date',
 ',',
 'datediff',
 '(',
 't_up',
 'minute',
 ',',
 'p',
 '.',
 'creationdate',
 ',',
 'a',
 '.',
 'creationdate',
 ')',
 'as',
 'qatimegap',
 'from',
 'posts',
 'as',
 'p',
 ',',
 'posts',
 'as',
 'a',
 ',',
 'users',
 'as',
 'u',
 'where',
 'p',
 '.',
 'i',
 'd',
 '=',
 'a',
 '.',
 'parentid',
 'and',
 'p',
 '.',
 'owneruserid',
 '=',
 'u',
 '.',
 'i',
 'd',
 'and',
 'a',
 '.',
 'creationdate',
 '=',
 '(',
 'select',
 'min',
 '(',
 'tau',
 '.',
 'creationdate',
 ')',
 'from',
 'posts',
 'tau',
 'where',
 'tau',
 '.',
 'parentid',
 '=',
 'p',
 '.',
 'i',
 'd

## Tokenize

In [161]:
np.percentile([len(o) for o in sql_tok], 90)

199.0

In [163]:
keep = np.array([len(o)<30 for o in sql_tok])

In [164]:
sql_tok = np.array(sql_tok)[keep]

In [167]:
# pickle.dump(sql_tok, (DATA_PATH/'SQL_tok.pkl').open('wb'))

In [170]:
sql_tok = pickle.load((DATA_PATH/'sql_tok.pkl').open('rb'))

In [173]:
sql_tok.shape

(5081,)

In [176]:
def toks2ids(tok,pre):
    freq = Counter(p for o in tok for p in o)
    itos = [o for o,c in freq.most_common(40000)]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    ids = np.array([([stoi[o] for o in p] + [2]) for p in tok])
    np.save(TMP_PATH/f'{pre}_ids.npy', ids)
    pickle.dump(itos, open(TMP_PATH/f'{pre}_itos.pkl', 'wb'))
    return ids,itos,stoi

In [177]:
en_ids,en_itos,en_stoi = toks2ids(sql_tok,'sql')


In [179]:
def load_ids(pre):
    ids = np.load(TMP_PATH/f'{pre}_ids.npy')
    itos = pickle.load(open(TMP_PATH/f'{pre}_itos.pkl', 'rb'))
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    return ids,itos,stoi

In [180]:
sql_ids,sql_itos,sql_stoi = load_ids('sql')


In [183]:
[sql_itos[o] for o in sql_ids[0]], len(sql_itos)

(['select',
  'i',
  'd',
  'from',
  'comments',
  'where',
  'lower',
  '(',
  'text',
  ')',
  'like',
  "'",
  '%',
  'accept',
  '%',
  'answer',
  '%',
  "'",
  'order',
  'by',
  'len',
  '(',
  'text',
  ')',
  'asc',
  '_eos_'],
 2342)

## Model

In [185]:
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [ ]:
def create_emb(vecs, itos, em_sz):
    emb = nn.Embedding(len(itos), em_sz, padding_idx=1)
    wgts = emb.weight.data
    miss = []
    for i,w in enumerate(itos):
        try: wgts[i] = torch.from_numpy(vecs[w]*3)
        except: miss.append(w)
    print(len(miss),miss[5:10])
    return emb

In [186]:
class Seq2SeqRNN(nn.Module):
    def __init__(self, vecs_enc, itos_enc, em_sz_enc, vecs_dec, itos_dec, em_sz_dec, nh, out_sl, nl=2):
        super().__init__()
        self.nl,self.nh,self.out_sl = nl,nh,out_sl
        self.emb_enc = create_emb(vecs_enc, itos_enc, em_sz_enc)
        self.emb_enc_drop = nn.Dropout(0.15)
        self.gru_enc = nn.GRU(em_sz_enc, nh, num_layers=nl, dropout=0.25)
        self.out_enc = nn.Linear(nh, em_sz_dec, bias=False)
        
        self.emb_dec = create_emb(vecs_dec, itos_dec, em_sz_dec)
        self.gru_dec = nn.GRU(em_sz_dec, em_sz_dec, num_layers=nl, dropout=0.1)
        self.out_drop = nn.Dropout(0.35)
        self.out = nn.Linear(em_sz_dec, len(itos_dec))
        self.out.weight.data = self.emb_dec.weight.data
        
    def forward(self, inp):
        sl,bs = inp.size()
        h = self.initHidden(bs)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        enc_out, h = self.gru_enc(emb, h)
        h = self.out_enc(h)

        dec_inp = V(torch.zeros(bs).long())
        res = []
        for i in range(self.out_sl):
            emb = self.emb_dec(dec_inp).unsqueeze(0)
            outp, h = self.gru_dec(emb, h)
            outp = self.out(self.out_drop(outp[0]))
            res.append(outp)
            dec_inp = V(outp.data.max(1)[1])
            if (dec_inp==1).all(): break
        return torch.stack(res)
    
    def initHidden(self, bs): return V(torch.zeros(self.nl, bs, self.nh))

In [187]:
def seq2seq_loss(input, target):
    sl,bs = target.size()
    sl_in,bs_in,nc = input.size()
    if sl>sl_in: input = F.pad(input, (0,0,0,0,0,sl-sl_in))
    input = input[:sl]
    return F.cross_entropy(input.view(-1,nc), target.view(-1))#, ignore_index=1)

In [188]:
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [189]:
rnn = Seq2SeqRNN(fr_vecd, fr_itos, dim_fr_vec, en_vecd, en_itos, dim_en_vec, nh, enlen_90)
learn = RNN_Learner(md, SingleModel(to_gpu(rnn)), opt_fn=opt_fn)
learn.crit = seq2seq_loss

NameError: name 'fr_vecd' is not defined

In [ ]:
learn.lr_find()
learn.sched.plot()

## Inference

In [ ]:
x,y = next(iter(val_dl))
probs = learn.model(V(x))
preds = to_np(probs.max(2)[1])

for i in range(180,190):
    print(' '.join([fr_itos[o] for o in x[:,i] if o != 1]))
    print(' '.join([en_itos[o] for o in y[:,i] if o != 1]))
    print(' '.join([en_itos[o] for o in preds[:,i] if o!=1]))
    print()

##  State Machine

## Generate text base on Inference and state machine